In [25]:
import nibabel as nib
import numpy as np
from sklearn.cluster import KMeans
import os
import cv2
import matplotlib.pyplot as plt

In [59]:
cluster_dir = '/Volumes/Elements/Clustering_T2/kmeans Clusters_3'
image_dir = '/Volumes/Elements/Clustering_T2/intensity_normalized'
for name in os.listdir(image_dir):
    # Load image
    # if name.endswith('.nii.gz') and not name.startswith('.'):
    try:
        image_path = os.path.join(image_dir,name)
        img = nib.load(image_path)
        img_np = img.get_fdata()
        
        # Load clusters label
        clusters_path = os.path.join(cluster_dir,name)
        clusters = nib.load(clusters_path)
        clusters_np = clusters.get_fdata()
        
        # match the corresponding cluster region
        mask_1 = (clusters_np == 1)
        mask_2 = (clusters_np == 2)
        mask_3 = (clusters_np == 3)
        cluster_1 = np.where(mask_1, img_np, 0)
        cluster_2 = np.where(mask_2, img_np, 0)
        cluster_3 = np.where(mask_3, img_np, 0)
        
        plt.figure(figsize=(14, 6))
        plt.hist(cluster_1.flatten(), bins=256, range =(1,256),edgecolor='black', alpha=0.5,color = 'red',label='Low intensity')
        plt.hist(cluster_2.flatten(), bins=256, range =(1,256),edgecolor='black', alpha=0.5,color = 'PaleGreen',label='Mediean intensity')
        plt.hist(cluster_3.flatten(), bins=256, range =(1,256),edgecolor='black', alpha=0.5,color = 'Blue',label='High intensity')
        plt.title("Intensity Histogram", fontsize=14)
        plt.xlabel("Intensity Value", fontsize=12)
        plt.ylabel("Frequency", fontsize=12)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.legend(loc='upper right', fontsize=12)
        # plt.show()
        plt.savefig(f"/Volumes/Elements/Clustering_T2/Hist-all-images/{name}.png", dpi=300, bbox_inches='tight')
        plt.close()
    except Exception as e:
        print(f'Error occured:{e}')